In [ ]:
install.packages("TSdist", dependencies = T)
install.packages("TSclust", dependencies=T)
install.packages("factoextra", dependencies = T)
install.packages("fpc", dependencies = T)
install.packages("NbClust", dependencies=T)
install.packages("ggplot2", dependencies=T)
install.packages("ggsignif", dependenncies=T)
install.packages("ggsci", dependencies=T)
install.packages("dtwclust", dependencies=T)

In [ ]:
library(TSdist)
library(factoextra)
library(fpc)
library(NbClust)

In [ ]:
pivot_table = read.csv("../input/perhourelectricitymeterreadingaggregation/perHourElectricityConsumptionEachBuildingAllYearPivot.csv")
myvar = names(pivot_table) %in% c("building_id","cluster","primary_use")
pivot_table <- pivot_table[!myvar]

In [ ]:
#calculate the similarity matrix
dtwDistance <- TSDatabaseDistances(na.omit(pivot_table), distance = "dtw")

In [ ]:
km.res <- eclust(na.omit(dtwDistance),k = 3, "kmeans", nstart = 25, graph = FALSE)
# Visualize k-means clusters
fviz_cluster(km.res, geom = "point", ellipse.type = "norm",
             palette = "jco", ggtheme = theme_minimal())

In [ ]:
NbClust(data = pivot_table, diss=dtwDistance,distance=NULL, method = "ward.D", min.nc = 2, max.nc=8)

In [ ]:
library(dtwclust)

In [ ]:
plot(tsclust(pivot_table, type="hierarchical", distance="dtw"))

In [ ]:
write(dtwDistance, file="dtwDistanceMatrix")

In [ ]:
hc <- hclust(dtwDistance, method = "ward.D")

In [ ]:
plot(hc)

In [ ]:
temp <- cutree(hc, k=c(2,3,4))

In [ ]:
temp <- as.data.frame(temp)

In [ ]:
temp

In [ ]:
library(dplyr)

In [ ]:
temp %>% group_by("3") %>%
    summarise(n = n_distinct("2"))

In [ ]:
write.csv(temp,"cuttree.csv")

In [ ]:
install.packages("BBmisc", dependencies=T)

In [ ]:
pivot_table_normalized = BBmisc::normalize(pivot_table,"standardize", range=c(0,1))

In [ ]:
pivot_table_normalized

In [ ]:
dtwDistanceNormalized = TSDatabaseDistances(na.omit(pivot_table_normalized), distance = "dtw")

In [ ]:
save(dtwDistanceNormalized, file="normalizedDtwDistanceMatrix.RData")

In [ ]:
save(dtwDistance, file="unnormalizedDtwDistance.RData")

In [ ]:
save(pivot_table_normalized, file="normalizedPivotTable.RData")

In [ ]:
plot(tsclust(pivot_table, type="hierarchical", k=2:6, distance = "dtw"))

In [ ]:
hc <- hclust(dtwDistanceNormalized, method="complete")

In [ ]:
plot(hc)

In [ ]:
hc <- eclust(pivot_table_normalized[-c(1100,779,1198,1089,1022,1169,1160,1149,1285),], k=3, k.max=8, FUNcluster = "pam", hc_metric="euclidean", verbose=TRUE)

In [ ]:
dtwNormalizedOutliersremoved = TSDatabaseDistances(pivot_table_normalized[-c(1100,779,1198,1089,1022,1169,1160,1149,1285),], distance="dtw")

In [ ]:
save(dtwNormalizedOutliersremoved, file="dtwNormalizedOutliersremoved.RData")

In [ ]:
res.kkm <- eclust(pivot_table_normalized[-c(1100,779,1198,1089,1022,1169,1160,1149,1285),],FUNcluster = "kmeans",nstart=25, verbose=TRUE)

In [ ]:
res.kkm

In [ ]:
res.kkm$nbclust

In [ ]:
res.kkm <- eclust(pivot_table_normalized[-c(1100,779,1198,1089,1022,1169,1160,1149,1285),],FUNcluster = "pam", verbose=TRUE)

In [ ]:
res.km <- eclust(pivot_table[-c(1100,779,1198,1089,1022,1169,1160,1149,1285),],"pam", verbose=TRUE)

In [ ]:
fviz_gap_stat(res.kkm$gap_stat)

In [ ]:
fviz_gap_stat(res.km$gap_stat)

In [ ]:
NbClust(data = pivot_table_normalized,distance="euclidean", method = "kmeans", min.nc = 2)